<h1><center>Classificação: Árvore de Decisão</center></h1>

Nesta aula iremos estudar um dos algoritmos clássicos de classificação, Árvore de Decisão. Você usará esse algoritmo para construir um modelo a partir de dados históricos de pacientes e sua resposta a diferentes medicamentos, para então prever a classe de um paciente desconhecido ou para encontrar um medicamento adequado para este novo paciente.

# Importando os pacotes necessários

In [1]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

# Download dos dados
O download dos dados pode ser realizado por meio do IBM Object Storage, disponível em:

https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/drug200.csv

##### Obs.: O arquivo deve ser salvo na mesma pasta onde este arquivo estiver salvo.

## Entendendo o conjunto de dados
Imagine que você é um médico pesquisador compilando dados para um estudo. Você coletou dados sobre um grupo de pacientes, todos sofrendo da mesma doença. Durante o tratamento, cada paciente foi administrado com um dos 5 medicamentos, Droga A, Droga B, Droga C, Droga X e Drogra Y.
<br>
<br>
Parte do seu trabalho é construir um modelo para descobrir qual droga pode ser apropriada para um futuro paciente com a mesma doença. Os conjuntos de características deste conjunto de dados são idade, sexo, pressão arterial e colesterol dos pacientes, e o alvo é o medicamento que cada paciente foi submetido.
<br>
<br>
É uma amostra de classificador binário e você pode usar a parte de treinamento do conjunto de dados para construir uma árvore de decisão, e depois usá-la para prever a classe de um paciente desconhecido, ou prescrevê-lo um medicamento.

Lendo os dados e transformando em um dataframe do pandas:

In [2]:
my_data = pd.read_csv("drug200.csv", delimiter=",")
my_data[0:5]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


### Prática
Qual o tamanho do dataframe?

In [3]:
# Escreva seu código aqui



## Pré-processamento

Usando <b>my_data</b> como os dados do Drug.csv lidos pelo pandas, declare as seguintes variáveis: <br>

<ul>
     <li> <b> X </b> como a <b>Matriz de características</b> (dados de my_data) </li>
     <li> <b> y </b> como o <b> vetor de resposta (destino) </b> </li>
</ul>

Remove a coluna que contém o nome de destino, pois ele não contém valores numéricos.

In [4]:
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.113999999999999],
       [28, 'F', 'NORMAL', 'HIGH', 7.797999999999999],
       [61, 'F', 'LOW', 'HIGH', 18.043]], dtype=object)

Como você pode imaginar, algumas características deste conjunto de dados são categóricas, como __Sex__ ou __BP__. Infelizmente, as árvores de decisão do Sklearn não lidam com variáveis categóricas. Mas ainda podemos converter esses recursos em valores numéricos. Podemos utilizar __pandas.get_dummies()__ para converter variáveis categóricas em variáveis dummy/indicadoras.

In [5]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1]) 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3]) 

X[0:5]


array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.113999999999999],
       [28, 0, 2, 0, 7.797999999999999],
       [61, 0, 1, 0, 18.043]], dtype=object)

Agora podemos preencher a variável de destino.

In [6]:
y = my_data["Drug"]
y[0:5]

0    drugY
1    drugC
2    drugC
3    drugX
4    drugY
Name: Drug, dtype: object

## Configurando a Árvore de Decisão
Nós vamos utilizar <b>train / test split</b> em nossa <b>árvore de decisão</b>. Vamos importar <b>train_test_split</b> de <b>sklearn.cross_validation</b>.

In [7]:
from sklearn.model_selection import train_test_split

Agora o <b>train_test_split</b> retornará 4 parâmetros diferentes. Vamos nomeá-los:<br>
X_trainset, X_testset, y_trainset, y_testset <br><br>
O <b>train_test_split</b> precisará dos parâmetros: <br>
X, y, test_size = 0.3 e random_state = 3. <br><br>
O <b>X</b> e <b>y</b> são os arrays necessários antes da separação, o <b>test_size</b> representa a proporção do conjunto de dados de teste e o <b>random_state</b> garante a aleatoridade da separação.

In [8]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

## Prática
Exiba o formato (shape) do X_trainset e y_trainset. Garanta que as dimensões são equivalentes.

In [9]:
# Escreva seu código aqui



Exiba o formato (shape) do X_testset e y_testset. Garanta que as dimensões são equivalentes.

In [10]:
# Escreva seu código aqui



## Modelagem
Primeiro criaremos uma instância do <b>DecisionTreeClassifier</b> chamado <b>drugTree</b>. <br>
Dentro do classificador, especifique `criterion = "entropy"` para que possamos ver o ganho de informação de cada nó.

In [11]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree # exibe os parâmetros default

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Em seguida, vamos ajustar o conjunto de dados com a matriz de características <b>X_trainset</b> e o vetor de resposta de treinamento <b>y_trainset</b>

In [12]:
drugTree.fit(X_trainset,y_trainset)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

## Predição
Vamos fazer algumas <b>prediçoes</b> usando o conjunto de dados de validação (teste) e salvar na variável chamada <b>predTree</b>.

In [13]:
predTree = drugTree.predict(X_testset)

Você pode exibir <b>predTree</b> e <b>y_testset</b> se quiser comparar visualmente a previsão com os valores reais.

In [14]:
print (predTree [0:5])
print (y_testset [0:5])


['drugY' 'drugX' 'drugX' 'drugX' 'drugX']
40     drugY
51     drugX
139    drugX
197    drugX
170    drugX
Name: Drug, dtype: object


## Avaliação
Em seguida, vamos importar <b>metrics</b> do sklearn e verificar a precisão do nosso modelo.

In [15]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("Precisão da Árvore de Decisão: ", metrics.accuracy_score(y_testset, predTree))

Precisão da Árvore de Decisão:  0.9833333333333333


__Accuracy classification score__ calcula a precisão do subconjunto: o conjunto de rótulos previsto para uma amostra deve corresponder exatamente ao conjunto correspondente de rótulos em y_true.

Na classificação multilabel, a função retorna a precisão do subconjunto. Se todo o conjunto de rótulos previstos para uma amostra corresponder estritamente ao conjunto real de rótulos, a precisão do subconjunto será 1.0; caso contrário, é 0.0.

## Prática
Você pode calcular a precisão sem o sklearn?

In [16]:
# Escreva seu código aqui


## Visualização
Vamos visualizar a árvore

**Obs.:** Apenas execute essa sequência de comandos se você não tiver instalado os módulos pydotplus e graphviz<br><br>
No terminal do Anaconda digite:

In [17]:
conda install pydotplus
conda install python-graphviz
conda install graphviz

SyntaxError: invalid syntax (<ipython-input-17-33ea36f4fd32>, line 1)

In [ ]:
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [ ]:
dot_data = StringIO()
filename = "drugtree.png"
featureNames = my_data.columns[0:5]
targetNames = my_data["Drug"].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

Esta aula foi desenvolvida com base no material disponibilizado por Saeed Aghabozorgi

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>